In [1]:
import json
import pandas as pd
import numpy as np
import os
from os import path
import time
import requests
import csv
import re
from collections import defaultdict

In [44]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [16]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result']

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def get_size(size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony-open-staking/validators_with_page?active=true&page=0&search=&size={}&sortOrder=desc&sortProperty=expectedReturns".format(size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['total']

In [6]:
def get_validator(page, size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony-open-staking/validators_with_page?active=false&page={}&search=&size={}&sortOrder=desc&sortProperty=expectedReturns".format(page, size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['validators']

In [7]:
def read_csv(csv_file) -> (dict, list):
    r = requests.get(csv_file)
    s = [x.decode(encoding) for x in r.content.splitlines()]
    d = defaultdict(list)
    v = []
    id_name = []
    for line in csv.reader(s):
        group = line[1].strip()
        id_num = re.sub(r'/\d+$', '', line[3].strip())
        address = line[7].strip()
        if group in groups and re.match('one1', address) != None:
            d[group].append(address)
            v.append(address)
            id_name.append(id_num)
    return d, v, id_name

In [8]:
def drop_duplicate(df):
    dp = df[(df['id_name'] != "") & (df.id_name.notnull())]
    dp = dp[dp.id_name.str.contains("www.ankr.com") == False]
    df = df.loc[list(set(df.index) - set(dp[dp.duplicated(subset = ['id_name'])].index))]
    return df

In [9]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [10]:
def getNewValidatorInfo(address, df):
    index = df[df['address'] == address].index[0]
    return df.loc[index]['name'], df.loc[index]['security-contact'], df.loc[index]['website']

In [11]:
bls_key = []
availability = []
name = []
epos_status = []
apr = []
delegation = []
committee = []
website = []
details = []
address = []
security = []
identity = []
boot_status = []
dels = defaultdict(int)
# get the accumualted reward in current block
block = getBlockNumber()
next_block = block + 1
validator_infos = getAllValidatorInformation()
for info in validator_infos:  
    for d in info['validator']['delegations']:
        del_address = d['delegator-address']
        reward = d['reward']/1e18
        dels[del_address] += reward
        
# # get the new accumulated reward in next block
print("-- Need to wait for the next block to get the reward changes --")
while block < next_block:
    block = getBlockNumber()
    
validator_infos = getAllValidatorInformation()
new_dels = defaultdict(int)
for info in validator_infos:
    if not info['metrics']:
        bls_key.append(np.nan)
    else:
        bls_key.append(len(info['metrics']['by-bls-key']))
    if not info['current-epoch-performance']:
        availability.append(np.nan)
    else:
        availability.append(info['current-epoch-performance']['current-epoch-signing-percent'])
    name.append(info['validator']['name'])
    epos_status.append(info['epos-status'])
    apr.append(info['lifetime']['apr'])
    delegation.append(len(info['validator']['delegations'][0]))
    committee.append(info['currently-in-committee'])
    website.append(info['validator']['website'])
    details.append(info['validator']['details'])
    address.append(info['validator']['address'])
    security.append(info['validator']['security-contact'])
    identity.append(info['validator']['identity'])    
    boot_status.append(info['booted-status'])
    for d in info['validator']['delegations']:
        del_address = d['delegator-address']
        reward = d['reward']/1e18
        new_dels[del_address] += reward
# get the reward the validator earned in that block
reward_dict = diffAndFilter(dels, new_dels)

-- Need to wait for the next block to get the reward changes --


In [12]:
# get most infos except reward and uptime
df = pd.DataFrame(list(zip( name, address, website, details, security, identity, epos_status, committee, apr, delegation, bls_key, boot_status)), \
                  columns =['name','address', 'website','details','security-contact','identity','epos-status', 'currently-in-committee', 'apr', 'delegator-num', 'bls-key-num', 'boot_status'])
df.reset_index(inplace = True, drop = True)
# get reward per block
reward = pd.DataFrame(reward_dict.items(), columns=['address', 'reward'])
df = df.join(reward.set_index("address"), on = 'address')
# get accumulated reward
acc_reward = pd.DataFrame(new_dels.items(), columns=['address', 'acc_reward'])
df = df.join(acc_reward.set_index("address"), on = 'address')


# get uptime
size = get_size(1)
pages = size // 100 + 1
# get the validator info
validator = []
for i in range(pages):
    val = get_validator(i,100)
    validator.extend(val)
uptime = []
address = []
active_nodes = []
elected_nodes = []
active = []
for i in validator:
    uptime.append(i['uptime_percentage'])
    address.append(i['address'])
    active_nodes.append(i['active_nodes'])
    elected_nodes.append(i['elected_nodes'])
uptime_df = pd.DataFrame(list(zip(uptime, address, active_nodes, elected_nodes)), columns = ['uptime-percentage','address', 'active_nodes', 'elected_nodes'])
df = df.join(uptime_df.set_index("address"), on = 'address')
df.to_csv('./csv/validator_info.csv', index = False)

print("-- Epos Status Summary --")
count = df.groupby('epos-status')['epos-status'].count().reset_index(name = 'count')
elected, eligible, ineligible = count['count'][0], count['count'][1], count['count'][2]
print("Currently elected: %s \nEligible to be elected next epoch: %s \nNot eligible to be elected next epoch: %s \n" % (elected, eligible, ineligible))

-- Epos Status Summary --
Currently elected: 193 
Eligible to be elected next epoch: 208 
Not eligible to be elected next epoch: 202 



In [13]:
csv_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTUUOCAuSgP8TcA1xWY5AbxaMO7OSowYgdvaHpeMQudAZkHkJrf2sGE6TZ0hIbcy20qpZHmlC8HhCw1/pub?gid=0&single=true&output=csv'
encoding = 'utf-8'
groups = ['team', 'p-ops', 'foundational nodes', 'p-volunteer', 'hackers', 'community', 'partners']
by_group, csv_validators, id_name = read_csv(csv_link)

In [14]:
id_df = pd.DataFrame(list(zip(csv_validators, id_name)), columns = ['address','id_name'])
id_df.drop_duplicates(subset = ['address'], inplace = True)
new_df = df.join(id_df.set_index('address'), on = 'address')
lst = []
for name, group in new_df.groupby('epos-status'):
    lst.append(drop_duplicate(group))
new = pd.concat(lst)
print("-- Epos Status Summary (Unique) --")
new_count = new.groupby('epos-status')['epos-status'].count().reset_index(name = 'count')
elected, eligible, ineligible = new_count['count'][0], new_count['count'][1], new_count['count'][2]
print("Currently elected: %s \nEligible to be elected next epoch: %s \nNot eligible to be elected next epoch: %s \n" % (elected, eligible, ineligible))


-- Epos Status Summary (Unique) --
Currently elected: 187 
Eligible to be elected next epoch: 205 
Not eligible to be elected next epoch: 185 



In [15]:
dp = new_df[(new_df['id_name'] != "") & (new_df.id_name.notnull())]
dp = dp[dp.id_name.str.contains("www.ankr.com") == False]
dp_count = dp.groupby('id_name')['id_name'].count().reset_index(name = 'count')
lst = dp_count[dp_count['count'] > 1].id_name.tolist()
dp['is_duplicated'] = dp['id_name'].apply(lambda x: True if any(i in x for i in lst) else False)
dp_address = dp[(dp['is_duplicated'])][['address', 'name', 'id_name', 'epos-status']].sort_values(by = 'id_name').reset_index(drop = True)
print("-- Duplicate Validators --")
dp_address.to_csv('./csv/duplicate_validators.csv', index = False)
print("Save duplicate validators to ./csv/duplicate_validators.csv")

-- Duplicate Validators --
Save duplicate validators to ./csv/duplicate_validators.csv


In [16]:
new_validators = [x for x in df['address'] if x not in csv_validators]
print("-- New Validators --")
print("New Validators: %d" % len(new_validators))
for i in new_validators:
    name, security, website = getNewValidatorInfo(i, df)
    print("Address: %s, Validator Name: %s, Security Contact: %s, Website: %s" %(i, name, security, website))


-- New Validators --
New Validators: 246
Address: one17y6w09aem8pj7t7cx28xa75jq3qzwlnv83xm5z, Validator Name: HARMONY 2500 SATS EOY, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one1lsvf8fms0emzvw45eazx855cdjh6x467fa4ad8, Validator Name: NunuxNode, Security Contact: , Website: 
Address: one1rqpu3209r90dacjvh2tv9c53wdnweecrtlr2hk, Validator Name: LH2, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one1n0t2sxc02ss8jmvcxyrptruu69a62xj35a75jc, Validator Name: LH2, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one15hvt2vq2hhh5parkgel5dcamhwcn2dlnu2kaec, Validator Name: Cabbage_autonode, Security Contact: Cabbage, Website: harmony.one
Address: one1g520fddr2y4skj7crgp607uzzammlyy6tqgrln, Validator Name: greedy_autonode, Security Contact: sperm whale, Website: thepirate-bay.org
Address: one1z2mk04tjw0ttascmvznu9pq7zpuz9e3l0qxssp, Validator Name: johnsmith node, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one1ehyhce5h9f62

Address: one1lr978ndz96y9m9yfq6jw6uxhfwlreqvcrk4myn, Validator Name: harmony autonode, Security Contact: Daniel-VDM, Website: harmony.one
Address: one10pk5pmg3tnxhwm5gpv95nqmzju4p44rqjlnk9a, Validator Name: NodeKing, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one1zxhmchnykrudcyqan0f6tru3tsh5mej6jg6y4f, Validator Name: Chip, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one19pd5yjhah2rt8hj4wdplyctzd2c50shhwhxv3q, Validator Name: harmony autonode, Security Contact: Daniel-VDM, Website: harmony.one
Address: one1w0f9fx3l2u34qq4u68k7gg307g0fqaq5f4rl8s, Validator Name: SIMNode, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one1j3mjr9mfd4k2lhp5nuhc9c9m0mw57qg80gfnqz, Validator Name: harmony autonode, Security Contact: Daniel-VDM, Website: harmony.one
Address: one1mzt202nuppau8xx7g378p2p5ek8egar7m0mzvx, Validator Name: li-auto-non-docker, Security Contact: li@harmony.one, Website: li@harmony.one
Address: one1tcdagh2wwlcnw5c2dwenz9k0t9cks

Address: one1rxnkpq087frx66yl6d5884rqdwpa250ja63qts, Validator Name: Thunderbolt, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one18dxy6eh2wsd2yqyr2tt6qrptrxl9pl44rt95fu, Validator Name: WeAreOne, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one10cuh4y3qjts7mz7nqf67rcms78gfv3jdpuqff5, Validator Name: harmony autonode, Security Contact: Daniel-VDM, Website: harmony.one
Address: one1el3arlc0dczjpr8crhufgszurt6hcmlm66evwa, Validator Name: ROZETKA, Security Contact: Daniel-VDM, Website: harmony.one
Address: one1080c4h7uev7u68s330vnxsfw7h6t86ufer8923, Validator Name: artkrpt, Security Contact: Daniel-VDM, Website: harmony.one
Address: one16xt3gl6l7q8afksksgwdmdahd5s75trgy6ut3k, Validator Name: blink  harmony autonode, Security Contact: blink, Website: harmony.one
Address: one13akr6683l3lfnfu44r3n9qnmq6la0fsy8r5kdv, Validator Name: SINTO, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one1jhl3w8z3lvmyngym7apveus58vk2mmjn08jrdq, Validator